# Приложение и практически реализации


#### Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter
from IPython.display import Audio

#### LPC Vowel Synthesis

##### **Step 1. Load and Define Parameters**
The code starts by defining key parameters like sample rate, duration, fundamental frequency, and formant frequencies to simulate a vowel sound.

- `sample_rate = 16000`: Sets the sample rate to 16 kHz, which is typical for speech processing.
- `duration = 0.5`: Specifies the length of the sound to be 0.5 seconds.
- `f0 = 140`: This is the fundamental frequency of the simulated vowel (140 Hz is typical for a male voice).
- `formants = [730, 1090, 2440]`: Defines the formant frequencies for vowel "A" (/ɑ/).
- `bandwidths = [60, 90, 110]`: Specifies the bandwidths for the formants, determining their sharpness.

##### **Step 2. Generate Glottal Pulse Train**
A glottal pulse train is generated using the fundamental frequency. This models the source sound produced by the vocal folds.

- `t = np.arange(0, duration, 1/sample_rate)`: Creates the time array.
- `glottal_pulse_train = np.sin(2 * np.pi * f0 * t)`: Generates a sine wave at the fundamental frequency to simulate the excitation source (glottal pulses).

##### **Step 3. Create Resonator for Each Formant**
A resonator is created for each formant, which models the effect of the vocal tract on the sound. Each formant acts as a bandpass filter at a specific frequency.

- The code uses formant frequencies and bandwidths to generate filters:
    - `w = 2 * np.pi * f / sample_rate` converts the formant frequency to angular frequency.
    - `r = np.exp(-np.pi * b / sample_rate)` calculates the pole radius (related to bandwidth).
    - `a = [1, -2*r*np.cos(w), r**2]`: Defines the filter coefficients for the formant.

##### **Step 4. Apply Each Filter**
The glottal pulse train is passed through each filter corresponding to the formants, shaping the sound to resemble the target vowel.

- `vowel_sound = lfilter(b, a, vowel_sound)`: Filters the glottal pulse train using the LPC filter created for each formant.

##### **Step 5. Normalize the Output**
After applying the filters, the synthesized signal is normalized to ensure it stays within the audible range.

- `vowel_sound /= np.max(np.abs(vowel_sound))`: Normalizes the vowel sound by dividing by its maximum absolute value.

##### **Step 6. Plot the Synthesized Signal**
The final vowel sound is plotted in both time domain and frequency domain.

- `plt.plot(vowel_sound[:500])`: Plots the first 500 samples of the sound waveform.
- `plt.magnitude_spectrum(vowel_sound, Fs=sample_rate)`: Plots the frequency spectrum of the synthesized vowel sound, showing the formant peaks.

### Vowel's Parameters:
- **Vowel "A" (/ɑ/)**:
```python
formants = [730, 1090, 2440]
bandwidths = [60, 90, 110]
```

- **Vowel "E" (/e/)**:
```python
formants = [660, 1700, 2400]
bandwidths = [50, 100, 120]
```

- **Vowel "I" (/i/)**:
```python
formants = [270, 2290, 3010]
bandwidths = [60, 100, 120]
```

- **Vowel "O" (/o/)**:
```python
formants = [570, 840, 2410]
bandwidths = [60, 80, 100]
```

- **Vowel "U" (/u/)**:
```python
formants = [300, 870, 2240]
bandwidths = [50, 70, 110]
```



In [ ]:
# Parameters for vowel synthesis
fundamental_freq = 120  # Fundamental frequency (pitch) in Hz
formants = [730, 1090, 2440]  # Formant frequencies (for vowel 'a') in Hz


bandwidths = [60, 90, 110]  # Bandwidths for formants in Hz
duration = 1.0  # Duration of the vowel sound in seconds
sample_rate = 16000  # Sampling rate in Hz

# Generate a time vector for the duration of the signal
t = np.arange(0, duration, 1 / sample_rate)

# Create an excitation signal (a train of impulses for voiced sounds)
excitation = np.zeros_like(t)
impulse_interval = int(sample_rate / fundamental_freq)
excitation[::impulse_interval] = 1  # Generate impulses at the pitch period

# Define the LPC filter coefficients for the formant frequencies and bandwidths
# Compute LPC poles using the resonance formula
lpc_coeffs = np.ones(1)  # Starting with a trivial coefficient

for formant, bandwidth in zip(formants, bandwidths):
    # Calculate the resonant frequency in radians per sample
    omega = 2 * np.pi * formant / sample_rate
    
    # Calculate the pole radius based on the bandwidth
    r = np.exp(-np.pi * bandwidth / sample_rate)
    
    # Calculate LPC coefficients for the current formant
    poles = np.array([r * np.exp(1j * omega), r * np.exp(-1j * omega)])
    poly = np.poly(poles)  # Convert poles to LPC coefficients (polynomial)
    
    # Convolve the new polynomial with the existing LPC coefficients
    lpc_coeffs = np.convolve(lpc_coeffs, poly)

# Apply the LPC filter to the excitation signal to synthesize the vowel sound
synthesized_vowel = lfilter([1], lpc_coeffs, excitation)

# Normalize the synthesized vowel
synthesized_vowel /= np.max(np.abs(synthesized_vowel))

# Plot the excitation signal and the synthesized vowel
plt.figure(figsize=(12, 6))

# Plot excitation signal
plt.subplot(2, 1, 1)
plt.plot(t[:1000], excitation[:1000])
plt.title('Excitation Signal (Impulse Train)')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

# Plot synthesized vowel signal
plt.subplot(2, 1, 2)
plt.plot(t[:1000], synthesized_vowel[:1000])
plt.title('Synthesized Vowel Sound')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

# Playback the signal
Audio(data=synthesized_vowel, rate=sample_rate)